In [1]:
%reset -f

import os
import numpy as np
import pandas as pd
import re
from datetime import datetime as DT
## sklearns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,\
    f1_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_validate
## tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import math
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

sMyPath = r'C:\UCSD_ML_Capstone'
os.chdir(sMyPath)

In [2]:
#### Import Data
##### Feature Generation #####
### Return the file names

datapath = sMyPath + r'\01_Input\01_03_ProcessedData\Tiingo_Stock_daily_Transformed'
AvaTickers_df = pd.read_csv(sMyPath + r"\01_Input\01_01_DataCodes\sNA_Select_Tickers.csv")
AvaTickers = AvaTickers_df[(AvaTickers_df.Sector=='Technology')&(AvaTickers_df.Industry=='Semiconductors')].Tickers.values


In [3]:
#### 0. Functions Define ####
def RandomForestCustomize(y, X, n_estimators = 100, max_depth=6,\
                      criterion = 'gini', ifcv=False, cvsize = 10):

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    
    RFClf = RandomForestClassifier(n_estimators=n_estimators,\
                                   max_depth=max_depth, criterion=criterion)
    RFClf.fit(X_train, y_train)
    
    y_test_pred = RFClf.predict(X_test)

    Conf_Mat_tmp = confusion_matrix(y_test, y_test_pred)
    Reports_tmp = classification_report(y_test, y_test_pred)
    Accu_tmp = accuracy_score(y_test, y_test_pred)
    try:
        F1_tmp = f1_score(y_test, y_test_pred)
    except ValueError:
        F1_tmp = 0

    if ifcv:
        cv_results = cross_validate(RFClf, X, y, cv=cvsize)
        cv_score = cv_results['test_score'].mean()
    else:
        cv_score = 0
        
    return([Accu_tmp, Conf_Mat_tmp, Reports_tmp, F1_tmp, cv_score])

def DNN_Customize(y, X, epochs = 100, batch_size = 500, patience = 10, 
                  loss='binary_crossentropy', test_size = 0.2, dropout_rate = 0.2):

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size)

    ## Prepare and train
    model = Sequential()
    model.add(Dense(128, input_shape=(X_train.shape[1],), activation = 'sigmoid' )  )
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation = 'sigmoid'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation = 'sigmoid'))
    # model.add(Dense(Train_y.shape[-1],activation='linear'))
    model.compile(loss=loss, optimizer='adam')
    # model.summary()
    callbacks = EarlyStopping(monitor = 'loss', patience=patience)

    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=batch_size, callbacks=callbacks )    
    
    y_test_pred = model.predict(X_test)
    
    # round predictions 
    y_test_pred = [round(x[0]) for x in y_test_pred]
    
    
    Conf_Mat_tmp = confusion_matrix(y_test, y_test_pred)
    Reports_tmp = classification_report(y_test, y_test_pred)
    Accu_tmp = accuracy_score(y_test, y_test_pred)
    try:
        F1_tmp = f1_score(y_test, y_test_pred)
    except ValueError:
        F1_tmp = 0   
        
    return([Accu_tmp, Conf_Mat_tmp, Reports_tmp, F1_tmp])

    
def standardize(x):
    return((x-x.mean())/x.std())

In [4]:
#### 1. Data clean and process ####
Var_Q = 5/100

TickerStart = 0
iTicker = TickerStart
Conf_Mat_df = list()
Reports_df = list()
Accu_df = list()
F1_df = list()
cv_score_df = list()
ifcv, cvsize = (True, 10)

In [5]:
S_time = DT.now()
iTicker=0

Loop_start = DT.now()

Ticker = AvaTickers[iTicker]

DataRaw = pd.read_pickle(datapath + r'\\'+ Ticker + r'_tsfresh.zip')
df = DataRaw.loc[:,[not bool(re.search('duplicate', x))\
                    for x in DataRaw.columns]].copy()

df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(axis = 1)

### 1. Data initialization
## Daily
# Return
Ret_D_clf2 = df.Ret_D.copy()
Ret_D_clf2[df.Ret_D >= 0] = 1
Ret_D_clf2[df.Ret_D < 0] = 0

Ret_D_clf3 = df.Ret_D.copy()
Ret_D_clf3[df.Ret_D >= df.Ret_D.mean() + df.Ret_D.std()] = 1
Ret_D_clf3[df.Ret_D <= df.Ret_D.mean() - df.Ret_D.std()] = 2
Ret_D_clf3[(df.Ret_D > df.Ret_D.mean() - df.Ret_D.std())\
           & (df.Ret_D<=df.Ret_D.mean() + df.Ret_D.std())] = 0

Ret_D_clfVaR = df.Ret_D.copy()
Ret_D_clfVaR[df.Ret_D > df.Ret_D.quantile(Var_Q)] = 0
Ret_D_clfVaR[df.Ret_D <= df.Ret_D.mean() - df.Ret_D.std()] = 1
# Volatility
Vol_D_clf2 = df.Vol_D.copy()
Vol_D_clf2[df.Vol_D >= df.Vol_D.mean()] = 1
Vol_D_clf2[df.Vol_D < df.Vol_D.mean()] = 0

Vol_D_clf3 = df.Vol_D.copy()
Vol_D_clf3[df.Vol_D >= np.quantile(df.Vol_D, 0.95)] = 1
Vol_D_clf3[df.Vol_D <= np.quantile(df.Vol_D, 0.05)] = 2
Vol_D_clf3[(df.Vol_D > np.quantile(df.Vol_D, 0.05))\
           & (df.Vol_D<=np.quantile(df.Vol_D, 0.95))] = 0

## Weekly
# Return
Ret_W_clf2 = df.Ret_W.copy()
Ret_W_clf2[df.Ret_W >= 0] = 1
Ret_W_clf2[df.Ret_W < 0] = 0

Ret_W_clf3 = df.Ret_W.copy()
Ret_W_clf3[df.Ret_W >= df.Ret_W.mean() + df.Ret_W.std()] = 1
Ret_W_clf3[df.Ret_W <= df.Ret_W.mean() - df.Ret_W.std()] = 2
Ret_W_clf3[(df.Ret_W > df.Ret_W.mean() - df.Ret_W.std())\
           & (df.Ret_W<=df.Ret_W.mean() + df.Ret_W.std())] = 0

Ret_W_clfVaR = df.Ret_W.copy()
Ret_W_clfVaR[df.Ret_W > df.Ret_W.quantile(Var_Q)] = 0
Ret_W_clfVaR[df.Ret_W <= df.Ret_W.mean() - df.Ret_W.std()] = 1
# Volatility
Vol_W_clf2 = df.Vol_W.copy()
Vol_W_clf2[df.Vol_W >= df.Vol_W.mean()] = 1
Vol_W_clf2[df.Vol_W < df.Vol_W.mean()] = 0

Vol_W_clf3 = df.Vol_W.copy()
Vol_W_clf3[df.Vol_W >= np.quantile(df.Vol_W, 0.95)] = 1
Vol_W_clf3[df.Vol_W <= np.quantile(df.Vol_W, 0.05)] = 2
Vol_W_clf3[(df.Vol_W > np.quantile(df.Vol_W, 0.05))\
           & (df.Vol_W<=np.quantile(df.Vol_W, 0.95))] = 0

## Monthly
# Return
Ret_M_clf2 = df.Ret_M.copy()
Ret_M_clf2[df.Ret_M >= 0] = 1
Ret_M_clf2[df.Ret_M < 0] = 0

Ret_M_clf3 = df.Ret_M.copy()
Ret_M_clf3[df.Ret_M >= df.Ret_M.mean() + df.Ret_M.std()] = 1
Ret_M_clf3[df.Ret_M <= df.Ret_M.mean() - df.Ret_M.std()] = 2
Ret_M_clf3[(df.Ret_M > df.Ret_M.mean() - df.Ret_M.std())\
           & (df.Ret_M<=df.Ret_M.mean() + df.Ret_M.std())] = 0

Ret_M_clfVaR = df.Ret_M.copy()
Ret_M_clfVaR[df.Ret_M > df.Ret_M.quantile(Var_Q)] = 0
Ret_M_clfVaR[df.Ret_M <= df.Ret_M.mean() - df.Ret_M.std()] = 1
# Volatility
Vol_M_clf2 = df.Vol_M.copy()
Vol_M_clf2[df.Vol_M >= df.Vol_M.mean()] = 1
Vol_M_clf2[df.Vol_M < df.Vol_M.mean()] = 0

Vol_M_clf3 = df.Vol_M.copy()
Vol_M_clf3[df.Vol_M >= np.quantile(df.Vol_M, 0.95)] = 1
Vol_M_clf3[df.Vol_M <= np.quantile(df.Vol_M, 0.05)] = 2
Vol_M_clf3[(df.Vol_M > np.quantile(df.Vol_M, 0.05))\
           & (df.Vol_M<=np.quantile(df.Vol_M, 0.95))] = 0

## Construct lag ys
df['Ret_D_clf2'] = Ret_D_clf2
df['Ret_D_clf3'] = Ret_D_clf3
df['Ret_D_clfVaR'] = Ret_D_clfVaR
df['Vol_D_clf2'] = Vol_D_clf2
df['Vol_D_clf3'] = Vol_D_clf3

df['Ret_W_clf2'] = Ret_W_clf2
df['Ret_W_clf3'] = Ret_W_clf3
df['Ret_W_clfVaR'] = Ret_W_clfVaR
df['Vol_W_clf2'] = Vol_W_clf2
df['Vol_W_clf3'] = Vol_W_clf3

df['Ret_M_clf2'] = Ret_M_clf2
df['Ret_M_clf3'] = Ret_M_clf3
df['Ret_M_clfVaR'] = Ret_M_clfVaR
df['Vol_M_clf2'] = Vol_M_clf2
df['Vol_M_clf3'] = Vol_M_clf3

## Construct ys
df['y_Ret_D_clf2'] = Ret_D_clf2.shift(-1)
df['y_Ret_D_clf3'] = Ret_D_clf3.shift(-1)
df['y_Ret_D_clfVaR'] = Ret_D_clfVaR.shift(-1)
df['y_Vol_D_clf2'] = Vol_D_clf2.shift(-1)
df['y_Vol_D_clf3'] = Vol_D_clf3.shift(-1)

df['y_Ret_W_clf2'] = Ret_W_clf2.shift(-5)
df['y_Ret_W_clf3'] = Ret_W_clf3.shift(-5)
df['y_Ret_W_clfVaR'] = Ret_W_clfVaR.shift(-5)
df['y_Vol_W_clf2'] = Vol_W_clf2.shift(-5)
df['y_Vol_W_clf3'] = Vol_W_clf3.shift(-5)

df['y_Ret_M_clf2'] = Ret_M_clf2.shift(-21)
df['y_Ret_M_clf3'] = Ret_M_clf3.shift(-21)
df['y_Ret_M_clfVaR'] = Ret_M_clfVaR.shift(-21)
df['y_Vol_M_clf2'] = Vol_M_clf2.shift(-21)
df['y_Vol_M_clf3'] = Vol_M_clf3.shift(-21)

df = df.dropna()

y_features = ['y_Ret_D_clf2', 'y_Ret_D_clf3', 'y_Ret_D_clfVaR','y_Vol_D_clf2','y_Vol_D_clf3',
              'y_Ret_W_clf2', 'y_Ret_W_clf3', 'y_Ret_W_clfVaR','y_Vol_D_clf2','y_Vol_D_clf3',
              'y_Ret_M_clf2', 'y_Ret_M_clf3', 'y_Ret_M_clfVaR','y_Vol_D_clf2','y_Vol_D_clf3']


#### 2. RUn the models ####
X = df.copy()
X = X.drop(y_features,axis = 1)
X_std_norm = abs(X.std()/X.mean())
X = X.loc[:,(~X_std_norm.isna())&(X_std_norm > 1/100000)]
X = X.apply(standardize, axis=0)

Conf_Mat_list = list()
Reports_list = list()
Accu_list = list()
F1_list = list()

iy = 0
for iy in range(len(y_features)):

    print(r"Running " + Ticker + r" " + str(iy+1) + r"/" + str(len(y_features)))
    y = df[y_features[iy]]

    [Accu_tmp, Conf_Mat_tmp, Reports_tmp, F1_tmp] = \
        DNN_Customize(y, X, epochs = 200, batch_size = 500, patience = 35, \
                  loss='binary_crossentropy', test_size = 0.2, dropout_rate = 0.2)

    Accu_list.append(Accu_tmp)
    Conf_Mat_list.append(Conf_Mat_tmp)
    Reports_list.append(Reports_tmp)
    F1_list.append(F1_tmp)


Accu_list = pd.Series(Accu_list)
Conf_Mat_list = pd.Series(Conf_Mat_list)
Reports_list = pd.Series(Reports_list)
F1_list = pd.Series(F1_list)


indexnames = ['D_PosiNega', 'D_BullBear', 'D_VaR', 'D_Vol2', 'D_Vol3',
              'W_PosiNega', 'W_BullBear', 'W_VaR', 'W_Vol2', 'W_Vol3',
              'M_PosiNega', 'M_BullBear', 'M_VaR', 'M_Vol2', 'M_Vol3']

Accu_list.name = Ticker + r'_Accuracy_Score'
Accu_list.index = indexnames
Conf_Mat_list.name = Ticker + r'_Confusion_Matrix'
Conf_Mat_list.index = indexnames
Reports_list.name = Ticker + r'_Clf_Report'
Reports_list.index = indexnames
F1_list.name = Ticker + r'_F1_Score'
F1_list.index = indexnames

# Output
if len(Conf_Mat_df) < 1:
    Accu_df = Accu_list
    Conf_Mat_df = Conf_Mat_list
    Reports_df = Reports_list
    F1_df = F1_list
else:
    Conf_Mat_df = pd.concat([Conf_Mat_df, Conf_Mat_list], axis=1)
    Reports_df = pd.concat([Reports_df, Reports_list], axis=1)
    Accu_df = pd.concat([Accu_df, Accu_list], axis=1)
    F1_df = pd.concat([F1_df, F1_list], axis=1)

## Estimate finish time
Loop_end = DT.now()
LoopTime = Loop_end-Loop_start
TotalTime = Loop_end-S_time
AvgTime = (TotalTime)/(iTicker -TickerStart +1)
EstEnd = DT.now() + (len(AvaTickers) - iTicker - TickerStart - 1)*AvgTime

# AvgSeries[iTicker] = AvgTime
# EstEndSeries[iTicker] = EstEnd
estreport = '\n Done No.'+ str(iTicker+1) + " " +Ticker+ ";\n" +\
              "Loop time:" + str(LoopTime) + "\n" +\
              "Total time:" + str(TotalTime) + "\n" +\
              "Average time:" + str(AvgTime) + "\n" +\
              "Estimate end:" + str(EstEnd) + '\n'

print(estreport)

File_object = open(sMyPath + r'\02_Code\02_02_Model_Development\estimatetime.txt',"a")
File_object.write(estreport)
File_object.close()   

Running ADI 1/15
Epoch 1/200
5/5 [==============================] - 0s 70ms/step - loss: 0.8528
Epoch 2/200
5/5 [==============================] - 0s 69ms/step - loss: 0.7032
Epoch 3/200
5/5 [==============================] - 0s 76ms/step - loss: 0.6718
Epoch 4/200
5/5 [==============================] - 0s 71ms/step - loss: 0.6592
Epoch 5/200
5/5 [==============================] - 0s 77ms/step - loss: 0.6349
Epoch 6/200
5/5 [==============================] - 0s 68ms/step - loss: 0.6190
Epoch 7/200
5/5 [==============================] - 0s 80ms/step - loss: 0.5960
Epoch 8/200
5/5 [==============================] - 0s 70ms/step - loss: 0.5850
Epoch 9/200
5/5 [==============================] - 0s 70ms/step - loss: 0.5571
Epoch 10/200
5/5 [==============================] - 0s 70ms/step - loss: 0.5237
Epoch 11/200
5/5 [==============================] - 0s 75ms/step - loss: 0.5087
Epoch 12/200
5/5 [==============================] - 0s 77ms/step - loss: 0.4862
Epoch 13/200
5/5 [==============

5/5 [==============================] - 0s 68ms/step - loss: 0.0235
Epoch 102/200
5/5 [==============================] - 0s 67ms/step - loss: 0.0263
Epoch 103/200
5/5 [==============================] - 0s 67ms/step - loss: 0.0253
Epoch 104/200
5/5 [==============================] - 0s 61ms/step - loss: 0.0259
Epoch 105/200
5/5 [==============================] - 0s 66ms/step - loss: 0.0237
Epoch 106/200
5/5 [==============================] - 0s 66ms/step - loss: 0.0243
Epoch 107/200
5/5 [==============================] - 0s 66ms/step - loss: 0.0234
Epoch 108/200
5/5 [==============================] - 0s 69ms/step - loss: 0.0274
Epoch 109/200
5/5 [==============================] - 0s 62ms/step - loss: 0.0269
Epoch 110/200
5/5 [==============================] - 0s 67ms/step - loss: 0.0257
Epoch 111/200
5/5 [==============================] - 0s 59ms/step - loss: 0.0273
Epoch 112/200
5/5 [==============================] - 0s 64ms/step - loss: 0.0347
Epoch 113/200
5/5 [=======================

5/5 [==============================] - 0s 54ms/step - loss: -1.5804
Epoch 70/200
5/5 [==============================] - 0s 62ms/step - loss: -1.5832
Epoch 71/200
5/5 [==============================] - 0s 63ms/step - loss: -1.6352
Epoch 72/200
5/5 [==============================] - 0s 60ms/step - loss: -1.6316
Epoch 73/200
5/5 [==============================] - 0s 63ms/step - loss: -1.6468
Epoch 74/200
5/5 [==============================] - 0s 63ms/step - loss: -1.6615
Epoch 75/200
5/5 [==============================] - 0s 57ms/step - loss: -1.6857
Epoch 76/200
5/5 [==============================] - 0s 61ms/step - loss: -1.7421
Epoch 77/200
5/5 [==============================] - 0s 63ms/step - loss: -1.7610
Epoch 78/200
5/5 [==============================] - 0s 64ms/step - loss: -1.7239
Epoch 79/200
5/5 [==============================] - 0s 59ms/step - loss: -1.7966
Epoch 80/200
5/5 [==============================] - 0s 60ms/step - loss: -1.8167
Epoch 81/200
5/5 [=======================

5/5 [==============================] - 0s 64ms/step - loss: -3.4246
Epoch 169/200
5/5 [==============================] - 0s 64ms/step - loss: -3.4327
Epoch 170/200
5/5 [==============================] - 0s 65ms/step - loss: -3.4002
Epoch 171/200
5/5 [==============================] - 0s 69ms/step - loss: -3.4688 0s - loss: -3.
Epoch 172/200
5/5 [==============================] - 0s 66ms/step - loss: -3.4876
Epoch 173/200
5/5 [==============================] - 0s 76ms/step - loss: -3.5292
Epoch 174/200
5/5 [==============================] - 0s 62ms/step - loss: -3.5037
Epoch 175/200
5/5 [==============================] - 0s 63ms/step - loss: -3.5316
Epoch 176/200
5/5 [==============================] - 0s 68ms/step - loss: -3.4716
Epoch 177/200
5/5 [==============================] - 0s 61ms/step - loss: -3.5028
Epoch 178/200
5/5 [==============================] - 0s 70ms/step - loss: -3.6265 0s - loss:
Epoch 179/200
5/5 [==============================] - 0s 63ms/step - loss: -3.5566
Epoc

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

5/5 [==============================] - 0s 29ms/step - loss: 0.5153
Epoch 3/200
5/5 [==============================] - 0s 31ms/step - loss: 0.4956
Epoch 4/200
5/5 [==============================] - 0s 29ms/step - loss: 0.4911
Epoch 5/200
5/5 [==============================] - 0s 30ms/step - loss: 0.4712
Epoch 6/200
5/5 [==============================] - 0s 29ms/step - loss: 0.4480
Epoch 7/200
5/5 [==============================] - 0s 33ms/step - loss: 0.4343
Epoch 8/200
5/5 [==============================] - 0s 32ms/step - loss: 0.4280
Epoch 9/200
5/5 [==============================] - 0s 33ms/step - loss: 0.4203
Epoch 10/200
5/5 [==============================] - 0s 34ms/step - loss: 0.4105
Epoch 11/200
5/5 [==============================] - 0s 31ms/step - loss: 0.3937
Epoch 12/200
5/5 [==============================] - 0s 29ms/step - loss: 0.3783
Epoch 13/200
5/5 [==============================] - 0s 31ms/step - loss: 0.3645
Epoch 14/200
5/5 [==============================] - 0s 30ms/

5/5 [==============================] - 0s 36ms/step - loss: 0.0260
Epoch 105/200
5/5 [==============================] - 0s 36ms/step - loss: 0.0264
Epoch 106/200
5/5 [==============================] - 0s 34ms/step - loss: 0.0255
Epoch 107/200
5/5 [==============================] - 0s 35ms/step - loss: 0.0275
Epoch 108/200
5/5 [==============================] - 0s 34ms/step - loss: 0.0227
Epoch 109/200
5/5 [==============================] - 0s 37ms/step - loss: 0.0273
Epoch 110/200
5/5 [==============================] - 0s 45ms/step - loss: 0.0264
Epoch 111/200
5/5 [==============================] - 0s 42ms/step - loss: 0.0262
Epoch 112/200
5/5 [==============================] - 0s 44ms/step - loss: 0.0265
Epoch 113/200
5/5 [==============================] - 0s 38ms/step - loss: 0.0243
Epoch 114/200
5/5 [==============================] - 0s 36ms/step - loss: 0.0243
Epoch 115/200
5/5 [==============================] - 0s 35ms/step - loss: 0.0268
Epoch 116/200
5/5 [=======================

Epoch 5/200
5/5 [==============================] - 0s 36ms/step - loss: 0.3512
Epoch 6/200
5/5 [==============================] - 0s 35ms/step - loss: 0.3323
Epoch 7/200
5/5 [==============================] - 0s 31ms/step - loss: 0.3162
Epoch 8/200
5/5 [==============================] - 0s 30ms/step - loss: 0.3007
Epoch 9/200
5/5 [==============================] - 0s 30ms/step - loss: 0.2841
Epoch 10/200
5/5 [==============================] - 0s 33ms/step - loss: 0.2685
Epoch 11/200
5/5 [==============================] - 0s 34ms/step - loss: 0.2539
Epoch 12/200
5/5 [==============================] - 0s 33ms/step - loss: 0.2336
Epoch 13/200
5/5 [==============================] - 0s 33ms/step - loss: 0.2138
Epoch 14/200
5/5 [==============================] - 0s 33ms/step - loss: 0.1884
Epoch 15/200
5/5 [==============================] - 0s 35ms/step - loss: 0.1703
Epoch 16/200
5/5 [==============================] - 0s 32ms/step - loss: 0.1516
Epoch 17/200
5/5 [===========================

5/5 [==============================] - 0s 33ms/step - loss: -0.9439
Epoch 107/200
5/5 [==============================] - 0s 30ms/step - loss: -0.9684
Epoch 108/200
5/5 [==============================] - 0s 29ms/step - loss: -0.9884
Epoch 109/200
5/5 [==============================] - 0s 31ms/step - loss: -0.9972
Epoch 110/200
5/5 [==============================] - 0s 37ms/step - loss: -0.9952
Epoch 111/200
5/5 [==============================] - 0s 34ms/step - loss: -0.9920
Epoch 112/200
5/5 [==============================] - 0s 40ms/step - loss: -1.0232
Epoch 113/200
5/5 [==============================] - 0s 34ms/step - loss: -1.0176
Epoch 114/200
5/5 [==============================] - 0s 31ms/step - loss: -1.0387
Epoch 115/200
5/5 [==============================] - 0s 29ms/step - loss: -1.0593
Epoch 116/200
5/5 [==============================] - 0s 31ms/step - loss: -1.0671
Epoch 117/200
5/5 [==============================] - 0s 30ms/step - loss: -1.0388
Epoch 118/200
5/5 [===========

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
Conf_Mat_df

D_PosiNega                     [[156, 79], [143, 180]]
D_BullBear    [[344, 80, 0], [37, 31, 0], [41, 25, 0]]
D_VaR                             [[480, 27], [47, 4]]
D_Vol2                           [[354, 71], [90, 43]]
D_Vol3          [[456, 44, 0], [28, 8, 0], [19, 3, 0]]
W_PosiNega                      [[197, 49], [48, 264]]
W_BullBear    [[349, 68, 0], [24, 42, 0], [11, 64, 0]]
W_VaR                            [[482, 16], [19, 41]]
W_Vol2                           [[379, 46], [94, 39]]
W_Vol3          [[454, 56, 0], [20, 3, 0], [24, 1, 0]]
M_PosiNega                      [[191, 30], [27, 310]]
M_BullBear    [[352, 54, 0], [16, 58, 0], [14, 64, 0]]
M_VaR                            [[464, 21], [14, 59]]
M_Vol2                           [[360, 65], [91, 42]]
M_Vol3          [[471, 29, 0], [25, 8, 0], [23, 2, 0]]
Name: ADI_Confusion_Matrix, dtype: object

In [7]:
Reports_df

D_PosiNega                  precision    recall  f1-score   ...
D_BullBear                  precision    recall  f1-score   ...
D_VaR                       precision    recall  f1-score   ...
D_Vol2                      precision    recall  f1-score   ...
D_Vol3                      precision    recall  f1-score   ...
W_PosiNega                  precision    recall  f1-score   ...
W_BullBear                  precision    recall  f1-score   ...
W_VaR                       precision    recall  f1-score   ...
W_Vol2                      precision    recall  f1-score   ...
W_Vol3                      precision    recall  f1-score   ...
M_PosiNega                  precision    recall  f1-score   ...
M_BullBear                  precision    recall  f1-score   ...
M_VaR                       precision    recall  f1-score   ...
M_Vol2                      precision    recall  f1-score   ...
M_Vol3                      precision    recall  f1-score   ...
Name: ADI_Clf_Report, dtype: object

In [8]:
Accu_df

D_PosiNega    0.602151
D_BullBear    0.672043
D_VaR         0.867384
D_Vol2        0.711470
D_Vol3        0.831541
W_PosiNega    0.826165
W_BullBear    0.700717
W_VaR         0.937276
W_Vol2        0.749104
W_Vol3        0.818996
M_PosiNega    0.897849
M_BullBear    0.734767
M_VaR         0.937276
M_Vol2        0.720430
M_Vol3        0.858423
Name: ADI_Accuracy_Score, dtype: float64

In [9]:
F1_df

D_PosiNega    0.618557
D_BullBear    0.000000
D_VaR         0.097561
D_Vol2        0.348178
D_Vol3        0.000000
W_PosiNega    0.844800
W_BullBear    0.000000
W_VaR         0.700855
W_Vol2        0.357798
W_Vol3        0.000000
M_PosiNega    0.915805
M_BullBear    0.000000
M_VaR         0.771242
M_Vol2        0.350000
M_Vol3        0.000000
Name: ADI_F1_Score, dtype: float64

In [10]:
cv_score_df

[]